<a href="https://colab.research.google.com/github/weagan/Tools/blob/main/Multilingual_Microsoft_Edge_TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q edge-tts pyarabic gradio-client pydub nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 2.6 MB/s eta 0:00:00


In [2]:
import tempfile
import edge_tts
from gradio_client import Client
import pyarabic.araby as araby
import asyncio
from pydub import AudioSegment
import nltk
import gradio as gr

# Download required NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
# Language and voice mapping dictionary
language_dict = {
  "English": {
    "Jenny": "en-US-JennyNeural",
    "Guy": "en-US-GuyNeural"
  },
  "Arabic": {
    "Hamed": "ar-SA-HamedNeural",
    "Fatima": "ar-AE-FatimaNeural"
  },
  "Irish": {
    "Colm": "ga-IE-ColmNeural",
    "Orla": "ga-IE-OrlaNeural"
  }
  # Additional languages can be added here as needed
}

In [4]:
# Initialize Tashkeel client for Arabic text processing
client = Client("MohamedRashad/arabic-auto-tashkeel")

Loaded as API: https://mohamedrashad-arabic-auto-tashkeel.hf.space ✔


In [5]:
async def text_to_speech_edge(text, language_code, speaker, tashkeel_checkbox=False):
  """Convert text to speech with 5-second pauses between sentences."""

  # Process Arabic text with Tashkeel if requested
  if language_code == "Arabic" and tashkeel_checkbox:
    text = client.predict(
      input_text=araby.strip_diacritics(text),
      api_name="/infer_shakkala"
    )

  # Validate language and speaker selection
  if language_code not in language_dict:
      raise ValueError(f"Selected language '{language_code}' is not supported.")

  selected_language_speakers = language_dict[language_code]
  if speaker not in selected_language_speakers:
      raise ValueError(f"Speaker '{speaker}' is not available for language '{language_code}'.")

  voice = selected_language_speakers[speaker]

  # Split text into sentences
  sentences = nltk.sent_tokenize(text)
  full_audio = AudioSegment.empty()
  silent_segment = AudioSegment.silent(duration=5000)  # 5-second pause

  # Generate audio for each sentence
  for i, sentence in enumerate(sentences):
    communicate = edge_tts.Communicate(sentence, voice)

    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp_sentence_file:
      tmp_sentence_path = tmp_sentence_file.name
      await communicate.save(tmp_sentence_path)

    sentence_audio = AudioSegment.from_mp3(tmp_sentence_path)
    full_audio += sentence_audio

    # Add silence between sentences (not after the last one)
    if i < len(sentences) - 1:
      full_audio += silent_segment

  # Save concatenated audio
  with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp_final_file:
    tmp_final_path = tmp_final_file.name
    full_audio.export(tmp_final_path, format="mp3")

  return text, tmp_final_path

In [6]:
# Example usage demonstrating the enhanced functionality
async def run_examples():
    # English example
    print("Generating English speech with pauses...")
    english_text = "Hello, this is a test. This sentence has a pause after it. And this is the third sentence."
    output_text_en, audio_path_en = await text_to_speech_edge(english_text, "English", "Jenny")
    print(f"English Audio saved to: {audio_path_en}")

    # Arabic example with Tashkeel
    print("\nGenerating Arabic speech with Tashkeel and pauses...")
    arabic_text = "السلام عليكم ورحمة الله وبركاته. كيف حالك اليوم؟ أتمنى لك يوما سعيدا."
    output_text_ar, audio_path_ar = await text_to_speech_edge(arabic_text, "Arabic", "Hamed", True)
    print(f"Arabic Audio saved to: {audio_path_ar}")

    # Irish example
    print("\nGenerating Irish speech with pauses...")
    irish_text = "Amárach beidh sé fliuch. Bhí mé sa bhaile inné."
    output_text_ir, audio_path_ir = await text_to_speech_edge(irish_text, "Irish", "Colm")
    print(f"Irish Audio saved to: {audio_path_ir}")

# Run the examples
await run_examples()

Generating English speech with pauses...
English Audio saved to: /tmp/tmp5i6o3i9m.mp3

Generating Arabic speech with Tashkeel and pauses...
Arabic Audio saved to: /tmp/tmpkruuy74w.mp3

Generating Irish speech with pauses...
Irish Audio saved to: /tmp/tmp7w3ie5im.mp3


In [7]:
# Helper function for dynamic speaker selection
def get_speakers_for_language(language):
    speakers = list(language_dict.get(language, {}).keys())
    if speakers:
        return gr.update(choices=speakers, value=speakers[0])
    else:
        return gr.update(choices=[], value=None)

# Initial speaker list for Irish (default)
initial_speakers = list(language_dict.get("Irish", {}).keys())

In [ ]:
# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Multilingual TTS with Sentence Pauses")

    with gr.Row():
        with gr.Column():
            text_input = gr.Textbox(label="Enter Text", lines=5,
                                    placeholder="Type your text here (sentences will have 5-second pauses between them)...")
            language_dropdown = gr.Dropdown(
                list(language_dict.keys()), label="Select Language", value="Irish"
            )
            speaker_dropdown = gr.Dropdown(
                choices=initial_speakers, label="Select Speaker",
                value=initial_speakers[0] if initial_speakers else None
            )
            tashkeel_checkbox = gr.Checkbox(label="Add Tashkeel (Arabic Only)", value=False)
            generate_button = gr.Button("Generate Speech")

        with gr.Column():
            output_text = gr.Textbox(label="Processed Text")
            audio_output = gr.Audio(label="Generated Speech (with 5s pauses between sentences)")

    # Dynamic speaker update based on language selection
    language_dropdown.change(
        get_speakers_for_language,
        inputs=language_dropdown,
        outputs=speaker_dropdown
    )

    # Connect the TTS function to the interface
    generate_button.click(
        fn=text_to_speech_edge,
        inputs=[text_input, language_dropdown, speaker_dropdown, tashkeel_checkbox],
        outputs=[output_text, audio_output]
    )

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5188d5597597413f5a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Summary

### Modifications Made:
1. **Enhanced `text_to_speech_edge` function** to:
   - Split input text into sentences using `nltk.sent_tokenize()`
   - Generate separate audio for each sentence
   - Insert 5-second silent pauses between sentences using `pydub.AudioSegment.silent(duration=5000)`
   - Concatenate all audio segments into a final output

2. **Dependencies added**:
   - `pydub` for audio manipulation
   - `nltk` for sentence tokenization
   - Required NLTK data (`punkt` and `punkt_tab`) downloaded

### Key Features:
- **Natural pauses**: 5-second silences between sentences for better listening experience
- **Multilingual support**: Works with English, Arabic (with optional Tashkeel), Irish, and other languages
- **Dynamic interface**: Gradio UI with language-speaker synchronization
- **Error handling**: Validates language and speaker selections

### Usage:
Call `text_to_speech_edge(text, language, speaker, tashkeel_checkbox)` to generate speech with automatic sentence segmentation and pauses.